# 기초 통계 분석 후 의미 있는 특징만 훈련

---

### 1순위
- wk
- cnt_dt

### 2순위
- npc_hongmun
- cnt_enter_raid
- cnt_enter_raid_light
- cnt_clear_raid
- cnt_clear_raid_light
- party_chat
- guild_chat

### 3순위
- item_hongmun
- get_money
- duel_cnt
- duel_win
- partybattle_cnt
- partybattle_win
- cnt_enter_inzone_solo
- cont_enter_inzone_skilled
- cnt_enter_bam
- cnt_clear_inzone_solo
- cnt_clear_inzone_skilled
- cnt_clear_bam
- normal_chat
- whisper_chat
- district_chat
- faction_chat

---

## 데이터 불러오기
---

In [155]:
# 기초 모듈 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [161]:
# 추가 모듈 임포트
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.contrib import layers
from sklearn.svm import LinearSVC

In [157]:
# 트레이닝 데이터 로드
x_data_pre = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_activity.csv').sort_values(by='acc_id', ascending=True)
y_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_label.csv').sort_values(by='acc_id', ascending=True)

## 데이터 선처리
---

In [158]:
column_list = [
    'acc_id', 'wk', 'cnt_dt', 
    'npc_hongmun', 'cnt_enter_raid', 'cnt_enter_raid_light', 'cnt_clear_raid', 'cnt_clear_raid_light', 
    'party_chat', 'guild_chat', 'item_hongmun', 'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt', 
    'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_skilled', 'cnt_enter_bam', 'cnt_clear_inzone_solo', 
    'cnt_clear_inzone_skilled', 'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat', 'faction_chat']

x_data_post = x_data_pre[column_list]

In [159]:
# 트레이닝데이터 아이디별로 그룹화 및 평균
# 데이터의 시계열 특성이 사라지지만, 빨리 단순 회귀 모형을 볼 수 있음
x_data_value = x_data_post.groupby(by='acc_id').mean().values
y_data_value = y_data.label.values

In [160]:
#트레이닝셋, 발리데이션셋 구분
x_train = x_data_value[0:int(0.7 * len(x_data_value)),:]
y_train = y_data_value[0:int(0.7 * len(x_data_value))]

x_cv = x_data_value[int(0.7 * len(x_data_value)):,:]
y_cv = y_data_value[int(0.7 * len(x_data_value)):]

## 머신러닝 모델
- decision tree
- random forest
- gradient boosting
- svm
- logistic regression
- neural network
---

### decision tree

In [36]:
tree = DecisionTreeClassifier(criterion='entropy', max_depth=10, random_state=0)

In [37]:
tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [38]:
print("훈련 세트 정확도: {:.3f}".format(tree.score(x_train, y_train)))

훈련 세트 정확도: 0.678


In [39]:
print("크로스발리데이션 세트 정확도: {:.3f}".format(tree.score(x_cv, y_cv)))

크로스발리데이션 세트 정확도: 0.661


### random forest

In [73]:
forest = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1, max_features=14, max_depth=20)

In [74]:
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=14, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [75]:
print("훈련 세트 정확도: {:.3f}".format(forest.score(x_train, y_train)))

훈련 세트 정확도: 0.799


In [76]:
print("크로스발리데이션 세트 정확도: {:.3f}".format(forest.score(x_cv, y_cv)))

크로스발리데이션 세트 정확도: 0.693


### svm

In [162]:
linear_svm = LinearSVC().fit(x_train, y_train)

In [163]:
print("훈련 세트 정확도: {:.3f}".format(linear_svm.score(x_train, y_train)))

훈련 세트 정확도: 0.506


In [164]:
print("크로스발리데이션 세트 정확도: {:.3f}".format(linear_svm.score(x_cv, y_cv)))

크로스발리데이션 세트 정확도: 0.508


### logistic regression

In [78]:
#한국어로 된 레이블을 숫자로 인코딩
idx2label = list(set(y_train))
label2idx = {c:i for i,c in enumerate(idx2label)}

#new y_train & y_cv
y_train = [label2idx[v] for v in y_train]
y_cv = [label2idx[v] for v in y_cv]

In [102]:
# 레이블 개수
num_feature = len(x_train[0])
learning_rate = 0.01
label = len(idx2label)
training_epoch = 100
batch_size = 100

In [91]:
X = tf.placeholder(tf.float32, [None, num_feature])
Y = tf.placeholder(tf.int32, [None])

In [92]:
#Y_one_hot
Y_one_hot = tf.one_hot(Y, label)

In [93]:
#Parameter
w = tf.Variable(tf.random_normal([num_feature, label]), name = "weight")
b = tf.Variable(tf.random_normal([label]), name = "bias")

In [94]:
logits = tf.matmul(X, w) + b

In [95]:
#Cost Function - softmax cross entropy
with tf.name_scope("cost") as scope:
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost)

In [96]:
#Train - Gradient Descent
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

In [97]:
#Prediction
prediction = tf.argmax(logits, -1)

In [98]:
#Accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y_one_hot,-1)), tf.float32))

In [107]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(len(x_train) / batch_size)
        
        for i in range(total_batch):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            
            for i in range(5):
                cost_val, _ = sess.run([cost, train], feed_dict={X:batch_x,Y:batch_y})
            avg_cost += cost_val/total_batch
        
        print("[epoch : {}]  cost {}".format(epoch, avg_cost))
    
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={X:x_train, Y:y_train})))

    print(sess.run(accuracy,feed_dict={X:x_cv,Y:y_cv}))

[epoch : 0]  cost 1.4657357103484012
[epoch : 1]  cost 1.2290026840141846
[epoch : 2]  cost 1.1977522001947671
[epoch : 3]  cost 1.1799477635111122
[epoch : 4]  cost 1.1685526439121794
[epoch : 5]  cost 1.1604759567975997
[epoch : 6]  cost 1.1552003817898893
[epoch : 7]  cost 1.1516440667424894
[epoch : 8]  cost 1.1487494274548116
[epoch : 9]  cost 1.1463659117903024
[epoch : 10]  cost 1.1443683845656258
[epoch : 11]  cost 1.1427172735759168
[epoch : 12]  cost 1.141343164529121
[epoch : 13]  cost 1.1401674933944437
[epoch : 14]  cost 1.1391742369106848
[epoch : 15]  cost 1.1383526750973296
[epoch : 16]  cost 1.1376482542072017
[epoch : 17]  cost 1.1370121169090273
[epoch : 18]  cost 1.1364319384098045
[epoch : 19]  cost 1.1358982714584895
[epoch : 20]  cost 1.1354037684202207
[epoch : 21]  cost 1.1349425795248587
[epoch : 22]  cost 1.1345101795026238
[epoch : 23]  cost 1.134103012084959
[epoch : 24]  cost 1.1337182434116102
[epoch : 25]  cost 1.1333536633423393
[epoch : 26]  cost 1.133

### Neural Network

In [108]:
#한국어로 된 레이블을 숫자로 인코딩
idx2label = list(set(y_train))
label2idx = {c:i for i,c in enumerate(idx2label)}

#new y_train & y_cv
y_train = [label2idx[v] for v in y_train]
y_cv = [label2idx[v] for v in y_cv]

In [141]:
# 레이블 개수
num_feature = len(x_train[0])
learning_rate = 0.01
training_epoch = 50
batch_size = 100
label = len(idx2label)

In [142]:
X = tf.placeholder(tf.float32, [None, num_feature])
Y = tf.placeholder(tf.int32, [None])
keep_prob = tf.placeholder(tf.float32)

In [143]:
#Y_one_hot
Y_one_hot = tf.one_hot(Y, label)

In [144]:
#Layer1
with tf.name_scope('layer1') as scope:
    w1 = tf.get_variable("weight1", [num_feature, 256], tf.float32, layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([256]), name = "bias1")
    
    layer1 = tf.nn.relu(tf.matmul(X,w1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob)

In [145]:
#Layer2
with tf.name_scope('layer2') as scope:
    w2 = tf.get_variable("weight2", [256, 128], tf.float32, layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([128]), name = "bias2")
    
    layer2 = tf.nn.relu(tf.matmul(layer1,w2) + b2)
    layer2 = tf.nn.dropout(layer2, keep_prob)

In [146]:
#Layer3
with tf.name_scope('layer3') as scope:
    w3 = tf.get_variable("weight3", [128, 64], tf.float32, layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([64]), name = "bias3")
    
    layer3 = tf.nn.relu(tf.matmul(layer2,w3) + b3)
    layer3 = tf.nn.dropout(layer3, keep_prob)

In [147]:
#Layer4
with tf.name_scope('layer4') as scope:
    w4 = tf.get_variable("weight4", [64, 32], tf.float32, layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([32]), name = "bias4")
    
    layer4 = tf.nn.relu(tf.matmul(layer3,w4) + b4)
    layer4 = tf.nn.dropout(layer4, keep_prob)

In [148]:
#hypothesis
with tf.name_scope('hypothesis') as scope:
    w5 = tf.get_variable("weight5", [32, label], tf.float32, layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([label]), name = "bias5")
    
    logits = tf.matmul(layer4, w5) + b5

In [149]:
#Cost Function - softmax cross entropy
with tf.name_scope("cost") as scope:
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost)
    tf.summary.scalar("cost", cost)

In [150]:
#Train - Gradient Descent
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

In [151]:
#Prediction
prediction = tf.argmax(logits, -1)

In [152]:
#Accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y_one_hot,-1)), tf.float32))
    tf.summary.scalar("accuracy", accuracy)

In [154]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(len(x_train) / batch_size)
        
        for i in range(total_batch):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            
            for i in range(5):
                cost_val, _ = sess.run([cost, train], feed_dict={X:batch_x,Y:batch_y, keep_prob:0.7})
            avg_cost += cost_val/total_batch
        
        print("[epoch : {}]  cost {}".format(epoch, avg_cost))
    
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={X:x_train, Y:y_train, keep_prob:1})))
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={X:x_cv, Y:y_cv, keep_prob:1})))

[epoch : 0]  cost 1.2931659366403314
[epoch : 1]  cost 1.239807273830686
[epoch : 2]  cost 1.2233716380596162
[epoch : 3]  cost 1.203854899065836
[epoch : 4]  cost 1.171324930872234
[epoch : 5]  cost 1.1294019323587419
[epoch : 6]  cost 1.1064151785203384
[epoch : 7]  cost 1.0884148300545569
[epoch : 8]  cost 1.0738230114323741


KeyboardInterrupt: 

In [140]:
tf.reset_default_graph()